# Build a song recommender system

In [2]:
import turicreate

# Load some music data

In [3]:
song_data = turicreate.SFrame('song_data.sframe/')

# Explore our data

In [4]:
song_data

user_id,song_id,listen_count,title,artist
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOAKIMP12A8C130995,1,The Cove,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Paco De Lucia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBXHDL12A81C204C0,1,Stronger,Kanye West
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBYHAJ12A6701BF1D,1,Constellations,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODACBL12A8C13C273,1,Learn To Fly,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODDNQT12A6D4F5F7E,5,Apuesta Por El Rock 'N'Roll ...,Héroes del Silencio
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODXRTY12AB0180F3B,1,Paper Gangsta,Lady GaGa
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFGUAY12AB017B0A8,1,Stacked Actors,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFRQTD12A81C233C0,1,Sehr kosmisch,Harmonia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOHQWYZ12A6D4FA701,1,Heaven's gonna burn youreyes ...,Thievery Corporationfeat. Emiliana Torrini ...


In [5]:
song_data[song_data['artist'] == 'Jack Johnson']

user_id,song_id,listen_count,title,artist
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOAKIMP12A8C130995,1,The Cove,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBYHAJ12A6701BF1D,1,Constellations,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOYHEPA12A8C13097F,8,Moonshine,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOZOBWN12A8C130999,1,Holes To Heaven,Jack Johnson
bd4c6e843f00bd476847fb75c47b4fb430a06856 ...,SOCZTMT12AF72A078E,1,Belle,Jack Johnson
169f9f4c68b62d1887c7c0ac99d10a79cfca5daf ...,SOERYLG12A6701F07F,1,Times Like These,Jack Johnson
169f9f4c68b62d1887c7c0ac99d10a79cfca5daf ...,SOKUTUM12A6701D9CD,1,Do You Remember,Jack Johnson
405d396ea64d75b5eaefaaf8ac836f45fa56af4d ...,SOOABBO12A6701DFDA,1,Flake,Jack Johnson
77cac5c3389010b64d01b30f725c24de4a5bb626 ...,SOJJDYI12A6701FC23,1,Holes To Heaven,Jack Johnson
3f9ed694a79835c921ef6d94acd28f876c1d901e ...,SOGCDYR12AC961854A,5,You And Your Heart,Jack Johnson


In [6]:
len(set(song_data['song_id']))

10000

In [7]:
len(set(song_data['user_id']))

66346

In [8]:
len(song_data['user_id'])

1116609

In [9]:
len(song_data['song_id'])

1116609

In [10]:
from collections import Counter

In [11]:
dups = [k for k,v in Counter(song_data['song_id']).items() if v>1]

In [12]:
dups[:10]

['SOAKIMP12A8C130995',
 'SOBBMDR12A8C13253B',
 'SOBXHDL12A81C204C0',
 'SOBYHAJ12A6701BF1D',
 'SODACBL12A8C13C273',
 'SODDNQT12A6D4F5F7E',
 'SODXRTY12AB0180F3B',
 'SOFGUAY12AB017B0A8',
 'SOFRQTD12A81C233C0',
 'SOHQWYZ12A6D4FA701']

In [13]:
song_data[song_data['song_id'] == 'SOAKIMP12A8C130995'] # multiple users listen to the same song

user_id,song_id,listen_count,title,artist,song
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOAKIMP12A8C130995,1,The Cove,Jack Johnson,The Cove - Jack Johnson
7a4b8e7d2905d13422418b4f48cc85100892e013 ...,SOAKIMP12A8C130995,6,The Cove,Jack Johnson,The Cove - Jack Johnson
33280fc74b168e2667a2da5c6ab4df4cc6edfb23 ...,SOAKIMP12A8C130995,1,The Cove,Jack Johnson,The Cove - Jack Johnson
71cf32f8871060079133fb15238ec8a38937e676 ...,SOAKIMP12A8C130995,1,The Cove,Jack Johnson,The Cove - Jack Johnson
a079310e7d04390d2f9e74a5d9aa6d84935d5a3c ...,SOAKIMP12A8C130995,2,The Cove,Jack Johnson,The Cove - Jack Johnson
fc514c0b52aa9494e2a0e0fa42f054a3c37bbe01 ...,SOAKIMP12A8C130995,1,The Cove,Jack Johnson,The Cove - Jack Johnson
b1f0aaff60c3b4bb5330c5ae0d0d7441602463d1 ...,SOAKIMP12A8C130995,1,The Cove,Jack Johnson,The Cove - Jack Johnson
aa11634a378907e8c1878092648aaba2d69b7862 ...,SOAKIMP12A8C130995,1,The Cove,Jack Johnson,The Cove - Jack Johnson
48c17b7e5dcbfeb389423bbfd6be7a4a7acac787 ...,SOAKIMP12A8C130995,1,The Cove,Jack Johnson,The Cove - Jack Johnson
a63ffb2ce5cd6f03a59430e5151e6ae5440b7ddf ...,SOAKIMP12A8C130995,1,The Cove,Jack Johnson,The Cove - Jack Johnson


In [14]:
dups_u = [k for k,v in Counter(song_data['user_id']).items() if v>1]

In [15]:
dups_u[:10]

['b80344d063b5ccb3212f76538f3d9e43d87dca9e',
 'bd4c6e843f00bd476847fb75c47b4fb430a06856',
 '8937134734f869debcab8f23d77465b4caaa85df',
 '969cc6fb74e076a68e36a04409cb9d3765757508',
 '4bd88bfb25263a75bbdd467e74018f4ae570e5df',
 'e006b1a48f466bf59feefed32bec6494495a4436',
 '9d6f0ead607ac2a6c2460e4d14fb439a146b7dec',
 '9bb911319fbc04f01755814cb5edb21df3d1a336',
 'c737ec8c1b16ce8e39115f4432c9a7fc21ec47a1',
 '45544491ccfcdc0b0803c34f201a6287ed4e30f8']

In [16]:
song_data[song_data['user_id'] == 'b80344d063b5ccb3212f76538f3d9e43d87dca9e'] # a user has listenned to multiple songs

user_id,song_id,listen_count,title,artist
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOAKIMP12A8C130995,1,The Cove,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Paco De Lucia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBXHDL12A81C204C0,1,Stronger,Kanye West
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBYHAJ12A6701BF1D,1,Constellations,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODACBL12A8C13C273,1,Learn To Fly,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODDNQT12A6D4F5F7E,5,Apuesta Por El Rock 'N'Roll ...,Héroes del Silencio
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODXRTY12AB0180F3B,1,Paper Gangsta,Lady GaGa
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFGUAY12AB017B0A8,1,Stacked Actors,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFRQTD12A81C233C0,1,Sehr kosmisch,Harmonia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOHQWYZ12A6D4FA701,1,Heaven's gonna burn youreyes ...,Thievery Corporationfeat. Emiliana Torrini ...


## Show the most popular songs in the dataset

In [17]:
song_data['song'].show()

Materializing SArray

In [18]:
import matplotlib.pyplot as plt

# Count the number of unique users in the data

In [19]:
users = song_data['user_id'].unique()

In [20]:
print(len(users))
print(type(users))

66346
<class 'turicreate.data_structures.sarray.SArray'>


# Create a song recommender

In [21]:
train_data,test_data = song_data.random_split(.8,seed=0)

In [22]:
song_count = Counter(song_data['song_id'])

In [23]:
sorted(song_count.items())

[('SOAAAGQ12A8C1420C8', 34),
 ('SOAACPJ12A81C21360', 54),
 ('SOAACSG12AB018DC80', 49),
 ('SOAAEJI12AB0188AB5', 31),
 ('SOAAFAC12A67ADF7EB', 98),
 ('SOAAFYH12A8C13717A', 60),
 ('SOAAJMQ12A6D4F7D17', 56),
 ('SOAAKPM12A58A77210', 80),
 ('SOAALWN12A6D4F7FDA', 47),
 ('SOAAMOW12AB018149B', 23),
 ('SOAAOYI12AB01831CE', 38),
 ('SOAAROC12A6D4FA420', 153),
 ('SOAARXR12A8C133D15', 19),
 ('SOAATHE12A8C13ADD6', 41),
 ('SOAATLI12A8C13E319', 88),
 ('SOAAUKC12AB017F868', 95),
 ('SOAAVUV12AB0186646', 436),
 ('SOAAWEE12A6D4FBEC8', 215),
 ('SOABGOB12A6701D1FA', 56),
 ('SOABHNP12A8AE46E82', 49),
 ('SOABHYV12A6D4F6D0F', 173),
 ('SOABJBU12A8C13F63F', 411),
 ('SOABJTC12A58A7DE0E', 101),
 ('SOABNLP12A6D4F87F4', 77),
 ('SOABNPW12A6D4FC9B5', 45),
 ('SOABOHM12AB018509C', 55),
 ('SOABOXV12AC3DF82F7', 38),
 ('SOABPQU12A58A78441', 45),
 ('SOABQTG12A6701F3DB', 60),
 ('SOABRAB12A6D4F7AAF', 59),
 ('SOABTTR12A6D4FC2EB', 164),
 ('SOABXNE12A8C13B818', 56),
 ('SOACBLB12AB01871C7', 112),
 ('SOACDMD12A67AD8332', 84),
 ('SOA

## Create a very simple popularity recommender

In [24]:
popularity_model = turicreate.popularity_recommender.create(train_data,
                                                           user_id = 'user_id',
                                                           item_id = 'song')

Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a rating column, specify the column name to be used as target

and use a method that allows the use of a target.

Preparing data set.

Data has 893580 observations with 66085 users and 9952 items.

Data prepared in: 1.46107s

893580 observations to process; with 9952 unique items.

## Use the popularity model to make some predictions

In [25]:
popularity_model.recommend(users=[users[0]])

user_id,song,score,rank
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Sehr kosmisch - Harmonia,4754.0,1
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Undo - Björk,4227.0,2
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,You're The One - DwightYoakam ...,3781.0,3
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Dog Days Are Over (RadioEdit) - Florence + The ...,3633.0,4
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Revelry - Kings Of Leon,3527.0,5
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Horn Concerto No. 4 in Eflat K495: II. Romance ...,3161.0,6
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Secrets - OneRepublic,3148.0,7
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Fireflies - CharttraxxKaraoke ...,2532.0,8
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Tive Sim - Cartola,2521.0,9
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Drop The World - LilWayne / Eminem ...,2053.0,10


In [26]:
popularity_model.recommend(users=[users[1]])

user_id,song,score,rank
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Sehr kosmisch - Harmonia,4754.0,1
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Undo - Björk,4227.0,2
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,You're The One - DwightYoakam ...,3781.0,3
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Dog Days Are Over (RadioEdit) - Florence + The ...,3633.0,4
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Revelry - Kings Of Leon,3527.0,5
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Horn Concerto No. 4 in Eflat K495: II. Romance ...,3161.0,6
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Secrets - OneRepublic,3148.0,7
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Hey_ Soul Sister - Train,2538.0,8
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Fireflies - CharttraxxKaraoke ...,2532.0,9
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Tive Sim - Cartola,2521.0,10


# Build a recommender with personalization

In [27]:
personalized_model = turicreate.item_similarity_recommender.create(train_data,
                                                                  user_id = 'user_id',
                                                                  item_id = 'song')

Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a rating column, specify the column name to be used as target

and use a method that allows the use of a target.

Preparing data set.

Data has 893580 observations with 66085 users and 9952 items.

...

| 1.81s                               | 100              | 9952            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 2.91943s

## Apply personalized model to make song recommendations

In [28]:
personalized_model.recommend(users=[users[0]])

user_id,song,score,rank
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Cuando Pase El Temblor -Soda Stereo ...,0.019450453611520622,1
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Fireflies - CharttraxxKaraoke ...,0.014473731701190654,2
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Love Is A Losing Game -Amy Winehouse ...,0.014286596041459303,3
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Marry Me - Train,0.014133471709031325,4
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Secrets - OneRepublic,0.013591665488023024,5
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Sehr kosmisch - Harmonia,0.013398789442502536,6
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Te Hacen Falta Vitaminas- Soda Stereo ...,0.01293028317964994,7
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,OMG - Usher featuringwill.i.am ...,0.012777828253232516,8
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Y solo se me ocurreamarte (Unplugged) - ...,0.012341127945826603,9
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,No Dejes Que... -Caifanes ...,0.012104249917543851,10


In [29]:
personalized_model.recommend(users=[users[1]])

user_id,song,score,rank
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Riot In Cell Block NumberNine - Dr Feelgood ...,0.03749999403953552,1
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Sei Lá Mangueira -Elizeth Cardoso ...,0.03316326439380646,2
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,The Stallion - Ween,0.032258063554763794,3
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Rain - Subhumans,0.031415924429893494,4
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,West One (Shine On Me) -The Ruts ...,0.030677199363708496,5
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Back Against The Wall -Cage The Elephant ...,0.030120477080345154,6
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Life Less Frightening -Rise Against ...,0.0284431129693985,7
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,A Beggar On A Beach OfGold - Mike And The ...,0.023002490401268005,8
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Audience Of One - RiseAgainst ...,0.01939384639263153,9
279292bb36dbfc7f505e36ebf038c81eb1d1d63e ...,Blame It On The Boogie -The Jacksons ...,0.018987342715263367,10


# Apply model to find similar songs in the data set

In [30]:
personalized_model.get_similar_items(['With Or Without You - U2'])

song,similar,score,rank
With Or Without You - U2,I Still Haven't FoundWhat I'm Looking For ...,0.04285717010498047,1
With Or Without You - U2,Hold Me_ Thrill Me_ KissMe_ Kill Me - U2 ...,0.033734917640686035,2
With Or Without You - U2,Window In The Skies - U2,0.03283584117889404,3
With Or Without You - U2,Vertigo - U2,0.03007519245147705,4
With Or Without You - U2,Sunday Bloody Sunday - U2,0.02713179588317871,5
With Or Without You - U2,Bad - U2,0.02517986297607422,6
With Or Without You - U2,A Day Without Me - U2,0.023715436458587646,7
With Or Without You - U2,Another Time AnotherPlace - U2 ...,0.020325183868408203,8
With Or Without You - U2,Walk On - U2,0.020202040672302246,9
With Or Without You - U2,Get On Your Boots - U2,0.019685029983520508,10


In [31]:
personalized_model.get_similar_items(['Chan Chan (Live) - Buena Vista Social Club'])

song,similar,score,rank
Chan Chan (Live) - BuenaVista Social Club ...,Murmullo - Buena VistaSocial Club ...,0.1881188154220581,1
Chan Chan (Live) - BuenaVista Social Club ...,La Bayamesa - Buena VistaSocial Club ...,0.1871921420097351,2
Chan Chan (Live) - BuenaVista Social Club ...,Amor de Loca Juventud -Buena Vista Social Club ...,0.18483412265777588,3
Chan Chan (Live) - BuenaVista Social Club ...,Diferente - Gotan Project,0.021459221839904785,4
Chan Chan (Live) - BuenaVista Social Club ...,Mistica - Orishas,0.020576119422912598,5
Chan Chan (Live) - BuenaVista Social Club ...,Hotel California - GipsyKings ...,0.019304990768432617,6
Chan Chan (Live) - BuenaVista Social Club ...,Nací Orishas - Orishas,0.019157111644744873,7
Chan Chan (Live) - BuenaVista Social Club ...,Gitana - Willie Colon,0.018796980381011963,8
Chan Chan (Live) - BuenaVista Social Club ...,Le Moulin - Yann Tiersen,0.018796980381011963,9
Chan Chan (Live) - BuenaVista Social Club ...,Criminal - Gotan Project,0.018779337406158447,10


# Compare the models quantitatively
We now formally compare the popularity and the personalized models using precision-recall curves. 

In [32]:
model_performance = turicreate.recommender.util.compare_models(test_data, [popularity_model, personalized_model], user_sample=.05)

compare_models: using 2931 users to estimate model performance
PROGRESS: Evaluate model M0


recommendations finished on 1000/2931 queries. users per second: 11639.5

recommendations finished on 2000/2931 queries. users per second: 12024.1


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.020470829068577272 | 0.006502593657148416 |
|   2    | 0.01842374616171958  | 0.011366187927088652 |
|   3    | 0.017172751051973186 | 0.014832702268730931 |
|   4    | 0.015864892528147382 | 0.018088948846369548 |
|   5    | 0.014602524735585135 | 0.020396668042522704 |
|   6    | 0.01444330717616285  | 0.02434893757821088  |
|   7    | 0.013988399863527819 | 0.027889488918148086 |
|   8    | 0.013476629136813372 | 0.030102944176639175 |
|   9    | 0.01338185678001441  | 0.03326509155577737  |
|   10   | 0.013101330603889487 | 0.03598830753385517  |
+--------+----------------------+----------------------+
[10 rows x 3 columns]

PROGRESS: Evaluate model M1


recommendations finished on 1000/2931 queries. users per second: 11773

recommendations finished on 2000/2931 queries. users per second: 10870.6


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.05185943364039577  | 0.015441715492892574 |
|   2    |  0.0446946434663937  | 0.02510200478470696  |
|   3    | 0.038780848402138045 | 0.03251581240322283  |
|   4    | 0.03488570453770046  | 0.038412285239306725 |
|   5    | 0.03159331286250431  | 0.042333582531667906 |
|   6    | 0.028545433867849447 | 0.045131266291480635 |
|   7    | 0.026563337719939586 | 0.04914396480264388  |
|   8    | 0.02482088024564995  | 0.05253047650542984  |
|   9    | 0.023313999772546373 |  0.055618319895218   |
|   10   | 0.02217673149095874  | 0.05901889975434442  |
+--------+----------------------+----------------------+
[10 rows x 3 columns]



The table shows that the personalized model provides much better performance.